# Rule Learner algorithm (PRISM): Notes

Use lecture slides, book chapters or the original paper to understand the PRISM algorithm. 

Read this notebook. Then, __IN A SEPARATE NOTEBOOK__, implement the algorithm. Implement it step-by-step, add explanations of each step, keep the code clean and modular. Finally, test the correctness of your algorithm on the contact lenses [dataset](contact_lenses.csv) discussed in class, to see if it produces the same rules.

If the PRISM algorithm is designed to cover the entire dataset, then the rules can be organized into a decision table, and used for classification. However, in this project, we are more interested in extracting small nuggets of knowledge: the most reliable rules - with significant coverage and high accuracy. 

## 1. Suggestions

- Advice: work on the  algorithm implementation in IDE (such as Pycharm). This will allow to fix bugs easily. 
- After your algorithm works, you can copy it step-by-step to a new notebook and explain each step.
- Use `pandas` and `numpy` whenever possible to make the program faster (See an example in Section 4 of this notebook).
- Use list comprehensions instead of loops - these run at least 10 times faster than native Python loops.
- __Important:__ you do not have to use the starter logic below. You can write your own code from scratch. The goal was to help you, not to constrain your creativity to my way of thinking about the problem.


## 2. Implementation notes

First, we need a datastructure to store each rule. 
Possible implementation of required data types is given below. You are free to either reuse it or create your own.

Each Rule consists of antecedent (Left Hand Side) and consequent (Right Hand Side). The LHS includes multiple conditions joined with AND, and RHS is a class label. The Rule also needs to store its accuracy and coverage.

In [2]:
class Rule:
    def __init__(self, class_label):
        self.conditions = []  # list of conditions
        self.class_label = class_label  # rule class
        self.accuracy = 0
        self.coverage = 0
        
    def add_condition(self, condition):
        self.conditions.append(condition)

    def set_params(self, accuracy, coverage):
        self.accuracy = accuracy
        self.coverage = coverage
        
    def to_filter(self):
        result = ""
        for cond in self.conditions:
            result += cond.to_filter() + " & "
        result += "(current_data[columns[-1]] == class_label)"
        return result
    
    def to_filter_no_class(self):
        result = ""
        for cond in self.conditions:
            result += cond.to_filter() + " & "
        result += "True"
        return result
    
    # Human-readable printing of this Rule
    def __repr__(self):
        return "If {} then {}. Coverage:{}, accuracy: {}".format(self.conditions, self.class_label,
                                                                 self.coverage, self.accuracy)

The list of conditions contains several objects of class _Condition_. 

Each condition includes the _attribute name_ and the _value_. 

If the _value_ is numeric, then the condition also includes an additional field `true_false` which means the following: 
- *if true_false == True then values are >= value* 
- *if true_false == False then values are < value*
- If *true_false is None*, then this condition is simply of form *categorical attribute = value*.

In [3]:
class Condition:
    def __init__(self, attribute, value, true_false = None):
        self.attribute = attribute
        self.value = value
        self.true_false = true_false
        
    def to_filter(self):
        result = ""
        if self is None:
            return result
        if self.true_false is None:
            result += '(current_data["' + self.attribute + '"]' + "==" + '"' + self.value + '")'
        elif self.true_false:
            result += '(current_data["' + self.attribute + '"]' + ">=" + self.value + ")"
        else:
            result += '(current_data["' + self.attribute + '"]' + "<" + self.value + ")"
        return result

    def __repr__(self):
        if self.true_false is None:
            return "{}={}".format(self.attribute, self.value)
        else:
            return "{}>={}:{}".format(self.attribute, self.value, self.true_false)

Next comes the `learn_one_rule` algorithm. The required parameters are the names of the columns, the current subset of data, and the class label (RHS of the rule we are trying to learn). The optional parameters are thresholds `min_coverage` and `min_accuracy`. I find that it is easier to treat `min_coverage` as an absolute number rather than the proportion of records covered by the rule. For normal datasets we can set this value to 30, for example, to prevent creating unreliable rules with insignificant support from data.

The algorithm returns a new rule and maybe a subset of data covered by the Rule or the remaining data (not covered by the rule). 

In [4]:
import numpy as np

def learn_one_rule(columns, data, class_label, prev_rule=None, min_coverage = 30, min_accuracy = 0.6):
    current_data = data.copy()

    current_rule = prev_rule
    current_accuracy = 0
    current_coverage = 0
    covered_subset = None
    
    if current_rule is not None:
        current_accuracy = current_rule.accuracy
        current_coverage = current_rule.coverage
    
    best_col = None
    best_val = None
    true_false = None

    for col in columns[:-1]:
        
        unique_vals = current_data[col].unique().tolist()
        
        for val in unique_vals:
            
            rule_filter = np.ones(len(current_data[columns[-1]]), dtype=bool)
            if current_rule is not None:
                rule_filter = eval(current_rule.to_filter_no_class())
            
            if isinstance(val, int) or isinstance(val, float):
                    
                bigger = sum(np.where((current_data[col] >= val) & rule_filter, 1, 0))
                smaller = sum(np.where((current_data[col] < val) & rule_filter, 1, 0))
                bigger_subset = current_data[(current_data[col] >= val) & rule_filter]
                smaller_subset = current_data[(current_data[col] < val) & rule_filter]
                
                maxi = max(bigger, smaller, min_coverage)
                if bigger ==  maxi or  smaller == maxi:
                    bigger_cov = sum(np.where(bigger_subset[columns[-1]] == class_label, 1, 0))
                    bigger_acc = bigger_cov/sum(bigger_subset[columns[-1]])
                    
                    smaller_cov = sum(np.where(smaller_subset[columns[-1]] == class_label, 1, 0))
                    smaller_acc = smaller_cov/sum(smaller_subset[columns[-1]])
                    
                    choose_bigger = True if bigger_acc > smaller_acc else False
                    if bigger_acc == smaller_acc:
                        if bigger_cov > smaller_cov:
                            choose_bigger = True
                else:
                    continue
                    
                
                if bigger_acc >= current_accuracy or smaller_acc >= current_accuracy:
                    best_col = col
                    best_val = val
                    current_accuracy = bigger_acc if choose_bigger else smaller_acc
                    current_coverage = bigger_cov if choose_bigger else smaller_cov
                    true_false = True if choose_bigger else False
              
            else:
                curr_subset = current_data[(current_data[col] == val) & rule_filter]
                total = len(curr_subset[columns[-1]])
                if total == 0:
                    continue

                curr_cov = len(curr_subset[curr_subset[columns[-1]] == class_label])
                curr_acc = curr_cov/total

                if curr_acc >= current_accuracy and curr_cov >= min_coverage:
                    best_col = col
                    best_val = val
                    current_accuracy = curr_acc
                    current_coverage = curr_cov
                    true_false = None

    if best_col is not None:

        if current_rule is None:
            current_rule = Rule(class_label)

        condition = Condition(best_col, best_val, true_false)
        current_rule.add_condition(condition)
        current_rule.set_params(current_accuracy, current_coverage)
        rule_filter = eval(current_rule.to_filter_no_class())

        if true_false is None:
            covered_subset = current_data[rule_filter]
        else:
            if true_false == True:
                covered_subset = current_data[rule_filter]
            else:
                covered_subset = current_data[rule_filter]
    
    return (current_rule, covered_subset)

Finally, the main algorithm `learn_rules` takes as parameters list of columns, with the last column representing the class label, and the original data in form of pandas dataframe. Optionally, you should be able to specify the list of classes that you are interested to explore first. Two optional threshold parameters `min_coverage` and `min_accuracy` set up the conditions of rule's validity for a given dataset.

In [5]:
import pandas as pd
import numpy as np

def learn_rules (columns, data, classes=None, 
                 min_coverage = 30, min_accuracy = 0.6):
    rules = []
    if classes is not None:
        class_labels = classes
    else:
        class_labels = data[columns[-1]].unique().tolist()

    current_data = data.copy()
    
    for class_label in class_labels:
        done = False
        while len(current_data) > min_coverage and not done:
            # Learn a rule with a single condition
            
            (rule, current_subset) = learn_one_rule(columns, current_data, class_label, None, min_coverage = 1)
            # If the best rule does not pass the coverage threshold - we are done with this class
            if rule is None:
                break

            # If we get the rule with coverage above threshold
            # We try to refine this rule
            if rule is not None:
                # try to improve the rule using the same learn_one_rule and passing existing rule as parameter
                # here need another loop which stops when accuracy is not improving
                prev_acc=0
                new_acc=rule.accuracy
                while(new_acc < 1.0 and new_acc > prev_acc):
                    (rule, current_subset) = learn_one_rule(columns, current_subset, class_label, rule, min_coverage, min_accuracy)
                    prev_acc = new_acc
                    new_acc = rule.accuracy
                # done with this rule
                if rule.accuracy >= min_accuracy:
                    rules.append(rule)
                    current_data = current_data.drop(current_data[eval(rule.to_filter())].index)
                else:
                    done = True
            else:
                done = True
                
    return rules

## 3. Correctness test  

Test your algorithm on the original dataset from the PRISM paper.

The dataset was downloaded from [here](https://archive.ics.uci.edu/ml/datasets/Lenses). The CSV version is included in this repository.

**Attribute Information**:

3 Classes:
- __1__ : the patient should be fitted with __hard__ contact lenses,
- __2__ : the patient should be fitted with __soft__ contact lenses,
- __3__ : the patient should __not__ be fitted with contact lenses.


Attributes:
1. age of the patient: (1) young, (2) pre-presbyopic, (3) presbyopic
2. spectacle prescription:  (1) myope, (2) hypermetrope
3. astigmatic:     (1) no, (2) yes
4. tear production rate:  (1) reduced, (2) normal

Presbyopia is physiological insufficiency of accommodation associated with the aging of the eye that results in progressively worsening ability to focus clearly on close objects. So "age=presbiopic" means old.

Hypermetropia: far-sightedness, also known as long-sightedness - cannot see close.
Myopia: nearsightedness - cannot see at distance.

In [6]:
import pandas as pd
import numpy as np

data_file = "contact_lenses.csv"
data = pd.read_csv(data_file, index_col=['id'])
data.columns

Index(['age', 'spectacles', 'astigmatism', 'tear production rate',
       'lenses type'],
      dtype='object')

We can replace numbers with actual values - for clarity.

In [7]:
# classes
conditions = [ data['lenses type'].eq(1), data['lenses type'].eq(2), data['lenses type'].eq(3)]
choices = ["hard","soft","none"]

data['lenses type'] = np.select(conditions, choices)

# age groups
conditions = [ data['age'].eq(1), data['age'].eq(2), data['age'].eq(3)]
choices = ["young","medium","old"]

data['age'] = np.select(conditions, choices)

# spectacles
conditions = [ data['spectacles'].eq(1), data['spectacles'].eq(2)]
choices = ["nearsighted","farsighted"]

data['spectacles'] = np.select(conditions, choices)

# astigmatism
conditions = [ data['astigmatism'].eq(1), data['astigmatism'].eq(2)]
choices = ["no","yes"]

data['astigmatism'] = np.select(conditions, choices)

# tear production rate
conditions = [ data['tear production rate'].eq(1), data['tear production rate'].eq(2)]
choices = ["reduced","normal"]

data['tear production rate'] = np.select(conditions, choices)

The test (do not run it before you finished the implementation of the rule learning algorithm):

In [8]:
column_list = data.columns.to_numpy().tolist()
rules = learn_rules (column_list, data, None, 1, 0.95)
for rule in rules[:20]:
    print(rule)

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0
If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0
If [astigmatism=no, age=medium] then soft. Coverage:1, accuracy: 1.0
If [astigmatism=no, age=young] then soft. Coverage:1, accuracy: 1.0
If [age=young] then hard. Coverage:2, accuracy: 1.0
If [spectacles=nearsighted, astigmatism=yes] then hard. Coverage:2, accuracy: 1.0


My results are given below for comparison:

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0

If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0

If [astigmatism=no, age=young] then soft. Coverage:1, accuracy: 1.0

If [astigmatism=no, age=medium] then soft. Coverage:1, accuracy: 1.0

If [age=young] then hard. Coverage:2, accuracy: 1.0

If [spectacles=nearsighted, astigmatism=yes] then hard. Coverage:2, accuracy: 1.0

## 4. Filtering data using conditions

Here is an example of how you could use the built-in `pandas` methods to produce a subset covered by a rule.
in the same way, you could also filter out rows covered by the rule, to obtain the remaining subset.

In [12]:
rule = Rule("soft")
cond1 = Condition("astigmatism","no")
cond2 = Condition("spectacles","farsighted")
rule.add_condition(cond1)
rule.add_condition(cond2)
rule.set_params(1, 3)
print(rule)

If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1


In [13]:
attr1 = cond1.attribute
val1 = cond1.value

attr2 = cond2.attribute
val2 = cond2.value

subset = data[(data[attr1]==val1) & (data[attr2]==val2) ]

print(subset)

       age  spectacles astigmatism tear production rate lenses type
id                                                                 
5    young  farsighted          no              reduced        none
6    young  farsighted          no               normal        soft
13  medium  farsighted          no              reduced        none
14  medium  farsighted          no               normal        soft
21     old  farsighted          no              reduced        none
22     old  farsighted          no               normal        soft


Copyright &copy; 2020 Marina Barsky. All rights reserved.